### 1. Постановка задачи

Создать модель, которая будет по загружаемой фотографии лица пользователя находить максимально похожих на него или неё  отечетвенных и иностранных знаменистостей, в зависимости от пола.

### 2. Получение данных в виде списка знаменитостей

Для получения баз данных лиц, на которых будет обучаться модель, необходим список популярных актеров и актрис. Получим его
из рейтинга ТОП 10000 популярных имён с сайта kinopoisk.ru, составленного на основе посещаемости страниц актёров и атрис, а также запросов к поисковой системе данного сайта.

Спарсим имена актёров и актрис из данного рейтинга. Для этого подключим необходимые библиотеки.

In [ ]:
from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import urllib
import requests
import json
import zlib
import time
import os

В процессе парсинга необходимо перебрать 50 страниц указанного рейтинга и получить с каждой из них следующие данные: место в рейтинге, идентификатор знаменитости на сайте кинопоиска, имя на русском и на английском языках. После чего заносим эти данные в таблицу Pandas и сохраняем полученную базу в CSV формате.

In [ ]:
data = []

for i in range(1, 51):
    rate_url = f'https://www.kinopoisk.ru/popular/names/page/{i}/'
    rate_res = requests.get(rate_url)
    soup = BeautifulSoup(rate_res.content, 'html.parser')
    
    divs = soup.find_all('div', class_='el')
    
    for div in divs:
        # место в рейтинге
        rate = div.find('b').text
        
        # получение kinopoinsk id
        href = div.find('a')['href']
        kp_id = href.split('/')[2]
        
        # получение русского имени
        rus_name = div.find_all('a')[1].text
        
        # получение анлийского имени
        try:
            eng_name = div.find('i').text
        except AttributeError:
            eng_name = None
            
        data.append([eng_name, rus_name, kp_id, rate])
        
    print('%d, parse url: %s' % (i, rate_url))
    
    time.sleep(1)

In [ ]:
df = pd.DataFrame(data, columns=['Eng_name', 'Rus_name', 'Kpoisk_id', 'Kpoisk_rate'])
df.head(10)

In [ ]:
# Сохранение данных в csv файл
df_path = './Documents/DataScience/PET-project/config'

if os.path.exists(df_path) != True:
    os.mkdir(df_path)

df.to_csv(df_path + '/celebs.csv')

Предварительный анализ полученной после парсинга базы данных знаменитостей показал, что в ней наряду с актёрами и актрисами, встречаются также имена лиц, хоть и имеющих прямое отношение к киноиндустрии, но менее известных и популярных. А именно сценаристов, режиссёров, актрис и актёров дубляжа, композиторов, продюсеров и операторов, как например, первая строка в рейтинге Лана Вачовски. Для данных лиц может быть затруднительно получение из общедоступных источников качественных изображений их лиц в достаточном количестве, что может негативно сказаться на обучении моделей в последующем.

Поэтому было принято решение парсить изображения только для актёров и актрис. Следовательно, базу данных нужно обогатить новыми данными о роли, которую играет каждое лицо в киноиндустрии.

Для этого будем парсить данные все того же сайта Кинопоиск, но теперь будем уже обращаться к его внутреннему приватному API, которое используется для отображения краткой информации о лице, при наведении курсора на его имя в рейтинге.

In [ ]:
df = pd.read_csv('./Documents/DataScience/PET-project/config/celebs.csv')
df['Role'] = ''
df.to_csv(df_path + '/celebs_ext.csv')

In [ ]:
df = pd.read_csv('./Documents/DataScience/PET-project/config/celebs_ext.csv')

In [ ]:
df.head()

In [ ]:
df_new = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df_new.to_csv(df_path + '/celebs_ext.csv')
df_new = pd.read_csv('./Documents/DataScience/PET-project/config/celebs_ext.csv')
df_new.head()

In [ ]:
def get_role(name):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0',
               'Accept': 'application/json',
               'Accept-Encoding': 'gzip, deflate, br',
               'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
               'Content-Type': 'application/json',
               'Host': 'www.kinopoisk.ru',
               'Connection': 'keep-alive',
               'Referer': 'https://www.kinopoisk.ru/popular/names/page/1/',
               'Sec-Fetch-Dest': 'empty',
               'Sec-Fetch-Mode': 'cors',
               'Sec-Fetch-Site': 'same-origin',
               'uber-trace-id': '1de025900d9f973b:b913feb825f292f2:0:1',
               'x-request-id': '1636134704015548-9903085037375331972',
               'X-Requested-With': 'XMLHttpRequest'}
    
    session = requests.Session()
    response = session.get('https://www.kinopoisk.ru/popular/names/page/1/')
    cookie_parsed = session.cookies.get_dict()
    
    cookie_str = ''
    for key in cookie_parsed:
        cookie_str += '%s=%s; ' % (key, cookie_parsed[key])
        cookie = {'Cookie': cookie_str}
        
    # получение исходного кода страниц по запросу к API
    request_url = f'https://www.kinopoisk.ru/api/tooltip/person/{name}/'
    request = urllib.request.Request(request_url, None, headers={**headers, **cookie})
    response = urllib.request.urlopen(request)
    
    try:
        unzip = zlib.decompress(response.read(), 16+zlib.MAX_WBITS)
        decoded = unzip.decode('utf8')
        parsed_json = json.loads(decoded)
    except:
        return None
    
    # получение данных о роли знаменитости в киноиндустрии
    try:
        role = parsed_json['person']['roles'][0]
    except IndexError:
        role = None
    
    print('Id: %s, роль: %s' % (name, role))
    
    return role

In [ ]:
df_new = pd.read_csv('./Documents/DataScience/PET-project/config/celebs_ext.csv')
df_new.head()
df_new.drop(['Unnamed: 0',  'role'], axis=1, inplace=True)
df_new.to_csv(df_path + '/celebs_ext.csv')
df_new = pd.read_csv('./Documents/DataScience/PET-project/config/celebs_ext.csv')

In [ ]:
df_new.drop(['role'], axis=1, inplace=True)

In [ ]:
df_new.head()

In [ ]:
for i in tqdm_notebook(df_new.query('Role.isnull()', engine='python').index):
    df_new.loc[i, 'Role'] = get_role(df_new.loc[i, 'Kpoisk_id'])
    # через каждые 100 запросов сохрняем результаты в файл, чтобы не потерять результаты при случайной остановке процесса
    if i % 100 == 0:
        df_new.to_csv(df_path + '/celebs_ext.csv')

In [ ]:
df_new.to_csv(df_path + '/celebs_ext.csv')
df_new.tail()

Посмотрим на графике сколько каких ролей в киноиндустрии представлено в полученной базе данных знаменитостей.

In [ ]:
df_new.groupby('Role').agg('Kpoisk_id').count().plot.bar(figsize=(12,8));

Как видно из графика более 80% данных в нем - это имена актёров и актрис. Посмотрим на распределение знаменитостей по признаку: отечественные или иностранные.

In [ ]:
# Иностранные знаменитости
df_new.query('Eng_name.notnull() & Role=="актер"', engine='python').shape,\
df_new.query('Eng_name.notnull() & Role=="актриса"', engine='python').shape

In [ ]:
# Отечественные знаменитости
df_new.query('Eng_name.isnull() & Role=="актер"', engine='python').shape,\
df_new.query('Eng_name.isnull() & Role=="актриса"', engine='python').shape

Таким образом, полученная база данных содержит имена и фамилии 2887 иностранных актёров и 2666 иностранных актрис, а также имена и фамилии 1463 отечественных актёров и 1294 актрис. Их изображения и будут использованы для обучения модели и последующей её работы для отображения максимально похожих на загруженное фото пользователем лиц.

### 3. Получение изображений для обучения моделей

В процессе работы с сайтом kinopoisk.ru было отмечено, что он содержит качественные и релевантные изображения актёров и актрис. Они могут составить основу для набора изображений, необходимого для обучения моделей. Поэтому, проведём ещё один парсинг кинопоиска через его внутренний API для получения и сохранения ссылок на данные изображения для их последующей загрузки.

Основное изображение знаменитостей занесём в базу данных в новую колонку под именем **Photo** и впоследствии будем считать их эталонными фото на конкретного атёра или актрисы. Остальные изображения, если они есть, занесём в базу данных в другую новую колонку **Gallery**, используя знак **;** в качестве разделителя.

Для снижения общего размера базы данных, повышения её удобночитаемости благодаря удалению однотипной информации, отбросим повторяющиеся части URL-адресов для колонок **Photo** и **Gallery**. Впоследствии отброшенные части можно бужет легко восстановить добавив к колонке **Photo** значение http://avatars.mds.yandex.net/get-kinopoisk-image/, а к **Gallery** добавив http://st.kp.yandex.net/images/kadr/{%num%}.jpg, заменив в нём *{%num%}* на номер изображения, сохранённый в базе данных.

In [ ]:
def get_photo(name):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:93.0) Gecko/20100101 Firefox/93.0',
               'Accept': 'application/json',
               'Accept-Encoding': 'gzip, deflate, br',
               'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
               'Content-Type': 'application/json',
               'Host': 'www.kinopoisk.ru',
               'Connection': 'keep-alive',
               'Referer': 'https://www.kinopoisk.ru/popular/names/page/1/',
               'TE': 'trailers',
               'Sec-Fetch-Dest': 'empty',
               'Sec-Fetch-Mode': 'cors',
               'Sec-Fetch-Site': 'same-origin',
               'uber-trace-id': '9cdcff51b5599475:29778de5f335b755:0:1',
               'x-request-id': '1636351355935334-7100532985104011688',
               'X-Requested-With': 'XMLHttpRequest'}
    
    session = requests.Session()
    response = session.get('https://www.kinopoisk.ru/popular/names/day/2021-11-07/page/7/')
    cookie_parsed = session.cookies.get_dict()
    
    cookie_str = ''
    for key in cookie_parsed:
        cookie_str += '%s=%s; ' % (key, cookie_parsed[key])
        cookie = {'Cookie': cookie_str}
        
    # Загрузка исходного кода страницы и получение ссылок на изображения
    request_url = f'https://www.kinopoisk.ru/api/tooltip/person/{name}/'
    request = urllib.request.Request(request_url, None, headers={**headers, **cookie})
    response = urllib.request.urlopen(request)
    
    try:
        unzip = zlib.decompress(response.read(), 16+zlib.MAX_WBITS)
        decoded = unzip.decode('utf8')
        parsed_json = json.loads(decoded)
    except:
        return None
    
    # получение основного изображения
    try:
        picture = parsed_json['person']['img']['posterMedium']['x2'][45:]
    except KeyError:
        picture = None
    
    # получение дополнительных изображений
    try:
        gallery = ';'.join([i['url'][34:].replace('.jpg', '') for i in parsed_json['person']['gallery']])
    except KeyError:
        gallery = None
        
    print('Id: %s, фото: %s' % (name, picture))

    return [picture, gallery]

In [ ]:
# Отбираем в базе данных всех актёров
df_new.query('Role=="актер"', engine='python')

In [ ]:
df_new['Photo'] = np.nan
df_new['Gallery'] = np.nan
df_new.head()

In [ ]:
# Загрузка данных об изображениях актёров для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
for i in tqdm_notebook(df_new.query('Photo.isnull() & Role=="актер"', engine='python').index):
    kp_res = get_photo(df_new.loc[i, 'Kpoisk_id'])
    try:
        df_new.loc[i, 'Photo'] = kp_res[0]
        df_new.loc[i, 'Gallery'] = kp_res[1]
        cnt += 1
        # через каждые 10 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 10 == 0:
            df_new.to_csv(df_path + '/celebs_extended.csv')
    except TypeError:
        print('Error get data!')

In [ ]:
df_new.to_csv(df_path + '/celebs_extended.csv')
df_new.head()

In [ ]:
# Загрузка данных об изображениях актрис для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
for i in tqdm_notebook(df_new.query('Photo.isnull() & Role=="актриса"', engine='python').index):
    kp_res = get_photo(df_new.loc[i, 'Kpoisk_id'])
    try:
        df_new.loc[i, 'Photo'] = kp_res[0]
        df_new.loc[i, 'Gallery'] = kp_res[1]
        cnt += 1
        # через каждые 10 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 10 == 0:
            df_new.to_csv(df_path + '/celebs_extended.csv')
    except TypeError:
        print('Error get data!')

In [ ]:
df_path = './Documents/DataScience/PET-project/config'
df_new = pd.read_csv('./Documents/DataScience/PET-project/config/celebs_extended.csv')
df_new.drop(['Unnamed: 0'], axis=1, inplace=True)
df_new.to_csv(df_path + '/celebs_extended.csv')
df_new.tail()

Посмотрим сколько знаменитостей у нас отпало на данном этапе из-за того, что мы не смогли найти для них эталонных изображения их лиц.

In [ ]:
# Иностранные знаменитости
df_new.query('Eng_name.notnull() & Role=="актер"', engine='python').shape[0] - \
df_new.query('Eng_name.notnull() & Photo.notnull() & Role=="актер"', engine='python').shape[0], \
df_new.query('Eng_name.notnull() & Role=="актриса"', engine='python').shape[0] - \
df_new.query('Eng_name.notnull() & Photo.notnull() & Role=="актриса"', engine='python').shape[0]

In [ ]:
# Отечественные знаменитости
df_new.query('Eng_name.isnull() & Role=="актер"', engine='python').shape[0] - \
df_new.query('Eng_name.isnull() & Photo.notnull() & Role=="актер"', engine='python').shape[0],\
df_new.query('Eng_name.isnull() & Role=="актриса"', engine='python').shape[0] - \
df_new.query('Eng_name.isnull() & Photo.notnull() & Role=="актриса"', engine='python').shape[0]

Теперь можно приступать к формированию базы данных лиц, на которых будет проводиться обучение моделей.

Создадим папку **sample_dataset**, в которую с разделением по полу знаменитостей и признаку отечественный/иностранный загрузим эталонные изображения лиц и сохраним их в отдельных папках по имени каждого актёра или актрисы.

In [ ]:
# функция загрузки таргет изображений знаменитостей

def get_sample_images(df, gender='men', natio='eng'):
    df_path = './Documents/DataScience/PET-project'
    
    # проверяем наличие папки sample_dataset и при необходимости создаём её
    if os.path.exists(f'{df_path}/sample_dataset/') != True:
        os.mkdir(f'{df_path}/sample_dataset/')
                 
    # проверяем наличие подпапки разделения по полу и при необходимости создаём её
    if os.path.exists(f'{df_path}/sample_dataset/{gender}') != True:
        os.mkdir(f'{df_path}/sample_dataset/{gender}')
        
    # проверяем наличие подпапки разделения по признаку отечественный/иностранный и при необходимости создаём её
    if os.path.exists(f'{df_path}/sample_dataset/{gender}/{natio}') != True:
        os.mkdir(f'{df_path}/sample_dataset/{gender}/{natio}')
        
    if natio == 'eng':
        req = 'Eng_name.notnull()'
        name = 'Eng_name'
    if natio == 'rus':
        req = 'Eng_name.isnull()'
        name = 'Rus_name'
        
    if gender == 'men':
        role = 'актер'
    if gender == 'women':
        role = 'актриса'
                 
    # пройдемся по каждому имени знаменитости
    for face in notebook.tqdm(list(df.query(f'{req} & Photo.notnull() & Role=="{role}"', engine='python')[f'{name}'].values)):
        face = face.strip()
        
        if os.path.exists(f'{df_path}/sample_dataset/{gender}/{natio}/{face}') != True:
            os.mkdir(f'{df_path}/sample_dataset/{gender}/{natio}/{face}')
        else:
            print(f'Images for {face} already uploaded!')
            continue
            
        ind = df.query(f'{name}=="{face}"').index
        img = df.iloc[ind[0]]['Photo']
        
        # загрузка фото с kinopoisk
        try:
            resp = requests.get(f'http://avatars.mds.yandex.net/get-kinopoisk-image/' + img)
        except TypeError:
            continue
            
        with open(f'{df_path}/sample_dataset/{gender}/{natio}/{face}/256.jpg', 'wb') as f:
            f.write(resp.content)
            
        print('%s picture parsed succefull!' % face)

In [ ]:
from tqdm import notebook

In [ ]:
# загрузка таргет изображений иностранных актёров
get_sample_images(df_new)

In [ ]:
# загрузка таргет изображений иностранных актрис
get_sample_images(df_new, gender='women')

In [ ]:
# загрузка таргет изображений отечественных актёров
get_sample_images(df_new, natio='rus')

In [ ]:
# загрузка таргет изображений отечественных актрис
get_sample_images(df_new, gender='women', natio='rus')

Создадим папку **gallery**, в которую с разделением по полу знаменитостей и признаку отечественный/иностранный загрузим галерею изображений, доступных на сайте kinopoisk, и сохраним их в отдельных папках по имени каждого актёра или актрисы.

In [ ]:
# функция загрузки галереи изображений знаменитостей с сайта kinopoisk

def get_gallery_images(df, gender='men', natio='eng'):
    df_path = './Documents/DataScience/PET-project'
    
    # проверяем наличие папки gallery и при необходимости создаём её
    if os.path.exists(f'{df_path}/gallery/') != True:
        os.mkdir(f'{df_path}/gallery/')
        
    # проверяем наличие подпапки разделения по полу и при необходимости создаём её
    if os.path.exists(f'{df_path}/gallery/{gender}') != True:
        os.mkdir(f'{df_path}/gallery/{gender}')
        
    # проверяем наличие подпапки разделения по признаку отечественный/иностранный и при необходимости создаём её
    if os.path.exists(f'{df_path}/gallery/{gender}/{natio}') != True:
        os.mkdir(f'{df_path}/gallery/{gender}/{natio}')
    
    if natio == 'eng':
        req = 'Eng_name.notnull()'
        name = 'Eng_name'
    if natio == 'rus':
        req = 'Eng_name.isnull()'
        name = 'Rus_name'
        
    if gender == 'men':
        role = 'актер'
    if gender == 'women':
        role = 'актриса'
        
    # пройдемся по каждому имени
    for face in notebook.tqdm(list(df.query(f'{req} & Gallery.notnull() & Role=="{role}"', engine='python')[f'{name}'].values)):
        face = face.strip()
        
        if os.path.exists(f'{df_path}/gallery/{gender}/{natio}/{face}') != True:
            os.mkdir(f'{df_path}/gallery/{gender}/{natio}/{face}')
        else:
            print(f'Images for {face} already uploaded!')
            continue
        
        ind = df.query(f'{name}=="{face}"').index
        gallery = df.iloc[ind[0]]['Gallery']
        
        try:
            nums = gallery.split(';')
        except AttributeError:
            continue
            
        i = 0
        for num in nums:
            i += 1
            # загрузка фото с kinopoisk
            try:
                resp = requests.get(f'http://st.kp.yandex.net/images/kadr/{num}.jpg')
            except TypeError:
                continue
            
            with open(f'{df_path}/gallery/{gender}/{natio}/{face}/Image_{i}.jpg', 'wb') as f:
                f.write(resp.content)
        
        print('%s :: gallery parsed succefull!' % face)

In [ ]:
# загрузка галереи изображений иностранных актёров
get_gallery_images(df_new)

In [ ]:
# загрузка галереи изображений иностранных актрис
get_gallery_images(df_new, gender='women')

In [ ]:
# загрузка галереи изображений отечественных актёров
get_gallery_images(df_new, natio='rus')

In [ ]:
# загрузка галереи изображений отечественных актрис
get_gallery_images(df_new, gender='women', natio='rus')

Произведём сжатие/растяжений эталонных изображений из папки **sample_dataset** до 256 px по меньшей стороне, как правило, это будет ширина, чтобы уменьшить место занимаемое изображениями на диске и провести их некоторую стандартизацию.

In [ ]:
SIZE = 256

In [ ]:
from PIL import Image
import glob

In [ ]:
# функция сжатия эталонных изображений знаменитостей

def image_compress(gender='men'):
    nation = ['eng', 'rus']
    for natio in nation:
        folders = glob.glob(f'./Documents/DataScience/PET-project/sample_dataset/{gender}/{natio}/*')
        
        for face in folders:
            face = face.strip()
            
            # выгрузим все файлы из подпапок
            i = f'{face}/256.jpg'
            
            try:
                # откроем изображение
                image = Image.open(i)
                # получим его размер
                size = image.size
                
                # определим наименьшую сторону
                if size[0] < size[1]:
                    min_size = size[0]
                else:
                    min_size = size[1]
                
                # получим коэффициент, на который нужно уменьшить/увеличить
                # изображение по одной из сторон до 256
                
                coef = SIZE / min_size
                # изменяем размер изображения
                resized_image = image.resize((int(size[0] * coef), int(size[1] * coef)))
                resized_image = resized_image.convert('RGB')
                
                # сохраняем изображение измененного размера
                resized_image.save(i)
            except FileNotFoundError:
                print('File: %s - not found!' % i)

In [ ]:
# сжатие эталонных изображений актёров
image_compress()

In [ ]:
# сжатие эталонных изображений актрис
image_compress(gender='women')

Произведём обрезку изображений для выделения лиц знаменитостей из папки **gallery** с последующий сжатием/растяжением для эконономии места на диске, занимаемом изображениями, стандартизации изображений и сокращения времени на последующее распознавание лиц.

In [ ]:
import face_recognition

In [ ]:
# функция для определения координат лиц на фотографии
def image_cropper(img):
    # загружаем при помощи face_recognition изображение
    image = face_recognition.load_image_file(img)
    # получаем координаты расположения лица (top-right, bottom-left)
    face_locations = face_recognition.face_locations(image)
    
    # если лицо не распозднается face_recognition, то удаляем это изображение
    if face_locations == []:
        os.remove(img)
        print('[%] Image: {} delete!'.format(img))
    else:
        face_width_coord_bigger = face_locations[0][1]
        face_width_coord_smaller = face_locations[0][3]
        
        face_height_coord_bigger = face_locations[0][2]
        face_height_coord_smaller = face_locations[0][0]
        
        face_width_frame = int((face_width_coord_bigger - face_width_coord_smaller) * 0.4)
        face_height_frame = int((face_height_coord_bigger - face_height_coord_smaller) * 0.6)
        
        # загрузка изображения
        image = Image.open(img)
        # получение размеров изображения
        size = image.size
        
        # получение ширины и высоты
        width = image.size[0]
        height = image.size[1]
        
        # вычисление новых координат для кропа
        # координата слева
        if face_width_coord_smaller - face_width_frame > 0:
            left = face_width_coord_smaller - face_width_frame
        else:
            left = 0
            
        # координата сверху
        if face_height_coord_smaller - face_height_frame > 0:
            top = face_height_coord_smaller - face_height_frame
        else:
            top = 0
            
        # координата справа
        if face_width_coord_bigger + face_width_frame < width:
            right = face_width_coord_bigger + face_width_frame
        else:
            right = width
            
        # координата снизу
        if face_height_coord_bigger + face_height_frame < height:
            bottom = face_height_coord_bigger + face_height_frame
        else:
            bottom = height
            
        # кроп изображения
        resized_image = image.crop((left, top, right, bottom))
        resized_image.save(img)

In [ ]:
# функция обрезки изображений знаменитостей из папки gallery для сохранения фрагментов с лицами

def image_compress(gender='men'):
    nation = ['eng', 'rus']
    for natio in nation:
        folders = glob.glob(f'./Documents/DataScience/PET-project/gallery/{gender}/{natio}/*')
        
        # пройдемся по каждому имени
        for face in notebook.tqdm(folders):
            face = face.strip()
            
            # выгрузим все название файлов из папки
            files = glob.glob(f'{face}/*')
            
            # пройдемся по списку файлов в цикле
            for img_file in files:
                try:
                    # обрезка изображений
                    image_cropper(img_file)
                    print('[%] Image: {} succefull cropped!\n'.format(img_file))
                except:
                    print('[!] Error crop image: {}\n'.format(img_file))
                    continue

In [ ]:
# обрезка галлереи изображений актёров
image_compress()

In [ ]:
# обрезка галлереи изображений актрис
image_compress(gender='women')

После выделения и сохранения в папке **gallery** из изображений фрагментов с лицами знаменитостей, полученные фотографии можно также сжать до 256 пикселей по наименьшей стороне.

In [ ]:
# функция сжатия/расширение изображений знаменитостей в папке gallery

def images_gallery_compress(gender='men'):
    nation = ['eng', 'rus']
    for natio in nation:
        folders = glob.glob(f'./Documents/DataScience/PET-project/gallery/{gender}/{natio}/*')
        
        for face in notebook.tqdm(folders):
            face = face.strip()
            
            # выгрузим все названия файлов из подпапок знаменитостей
            files = glob.glob(f'{face}/*')
            
            for img_file in files:
                # обрезка изображений
                try:
                    try:
                        # откроем изображение
                        image = Image.open(img_file)
                        # получим его размер
                        size = image.size
                    except:
                        continue
                    
                    # определим его наименьшую сторону
                    if size[0] < size[1]:
                        min_size = size[0]
                    else:
                        min_size = size[1]
                    
                    # получим коэффициент, на который нужно уменьшить/увеличить
                    # изображение по одной из сторон до 256
                    coef = SIZE / min_size
                    
                    # изменяем размер изображения
                    resized_image = image.resize((int(size[0] * coef), int(size[1] * coef)))
                    resized_image = resized_image.convert('RGB')
                    # сохраняем изображение в новом размере
                    resized_image.save(img_file)
                except FileNotFoundError:
                    print('File: %s - not found!' % i)

In [ ]:
# сжатие/растяжение галлереи изображений актёров
images_gallery_compress()

In [ ]:
# сжатие/растяжение галлереи изображений актрис
images_gallery_compress(gender='women')

В результате проведенных операций по сжатию и обрезке изображений объём дискового пространства, занимаемого фотографиями лиц знаменитостей, удалось сократить с примерно 9 Гб до 500 Мб.

Анализ полученных изображений показал, что в галерею достаточно часто попадают не самые лучшие изображения лиц. Например, они могут быть сфотографированы с очень сильно повёрнутой в любую сторону головой, прикрыты солнцезащитными очками и другими предметами или руками и так далее. При этом, в Кинопоиске содержатся дополнительные галереи знаменитостей из которых могут быть получены изображения лиц подходящего для обучения моделей качества.

Принято решение добавить эти изображения к уже полученым для формирования качественного датасета изображений лиц знаменитостей для последующего обучения моделей. Страницы с этими изображениями находятся по URL-адресам вида https://www.kinopoisk.ru/name/{id}/photos/, где {id} id-идентификатор знаменитости на Кинопоиске. В результате парсинга страниц будут получены ссылки изображения в виде https://avatars.mds.yandex.net/get-kinopoisk-image/1900788/b0bd9fe2-f3e3-4133-afcc-fa0eac0fc7fc/1920x. Части URL изображения в начале **https://avatars.mds.yandex.net/get-kinopoisk-image** и **1920x** в конце являются общими для каждого изображения и могут быть отброшены при сохранении в базу.

В качестве разделителя при сохранении изображений в датафрейм принято решение использовать двойную точку с запятой **;;** в новую колонку **Img_urls**. Если в процессе парсинга будет обнаружено, что для конкретной знаменитости существует несколько страниц с дополнительными изображениями, то номера этих страниц будут также сохранены в новую колонку **Img_pages** с разделителем в виде одинарной точки с запятой **;** для последующего парсинга с них дополнительных изображений.

In [ ]:
import random
import re

In [ ]:
cookies = ['i=za6sI7RI5Cp1qewNd8XHCd8ZSc2yPGHP3DZSDaf6YdBJBrZfxJ39j8I5etbsE59ijMG3+v1YXi0ePk0tXyUifGalOcQ=;',
           ' mda_exp_enabled=1; gdpr=0; _ym_uid=1622705461262579387;', 
           ' _ym_d=1640004873;',
           ' ya_sess_id=3:1639984242.5.0.1589825517989:FMaRJQ:8.1|9572252.0.2|30:203735.635399.N-c8fHyDYIpoqQ1_R5jORt-6nJY;',
           ' yandex_login=olegoff; yandexuid=1611715171589748827; mda2_beacon=1639984242768;',
           ' my_perpages=%5B%5D; users_info[check_sh_bool]=none; tc=431; mobile=no;',
           ' cycada=PmEQEPRadGGzAZU1ExaQmSga1Sjs0ExfG9qXqqDsBeQ=; is_gdpr=0;',
           ' PHPSESSID=j40blgdb3htj1irab9el8ktsu7; yandex_gid=35; uid=25438931;',
           ' _csrf_csrf_token=EX617llh6bIoKxiiziaQj3VmF1JEHrXW6SOEW2wsMRA;',
           ' desktop_session_key=85266d0f4c3cc027a6006d6805916b0bd3c030169a06eb72be43b5a8c1cc1e865d0676eda8be326db81bd192',
           'a53379f07ff352a16fee166aa1853f69c9caf0e6ba87778221b8c02be4891e2e8380a1fb5afd955a410baeaacd98d7a8b17b77d3;',
           ' desktop_session_key.sig=_xzV3qLXTFGBUE7VdgVkTKzKQ8M; _csrf=yqrOt6hMtn0xf6sRbKxM91SM;',
           ' ys=udn.cDpvbGVnb2Zm#c_chck.1387243862; _ym_isad=2; user_country=ru']

In [ ]:
headers = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
           'Cache-Control': 'max-age=0',
           'Connection': 'keep-alive',
           'Cookie': ''.join(cookies),
           'Host': 'www.kinopoisk.ru',
           'Referer': 'https://www.kinopoisk.ru/picture/2131681/',
           'Sec-Fetch-Dest': 'document',
           'Sec-Fetch-Mode': 'navigate',
           'Sec-Fetch-Site': 'same-origin',
           'Sec-Fetch-User': '?1',
           'TE': 'trailers',
           'Upgrade-Insecure-Requests': '1',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:95.0) Gecko/20100101 Firefox/95.0'}

In [ ]:
# функция загрузки дополнительных изображений знаменитостей с сайта kinopoisk
def get_additional_images_urls(name, p=1, headers=headers, cookies=cookies):
    # Загрузка исходного кода страницы и получение ссылок на изображения
    if p == 1:
        request_url = f'https://www.kinopoisk.ru/name/{name}/photos/'
    else:
        request_url = f'https://www.kinopoisk.ru/name/{name}/photos/page/{p}/'

    try:
        request = urllib.request.Request(request_url, None, headers={**headers})
        response = urllib.request.urlopen(request)

        unzip = zlib.decompress(response.read(), 16+zlib.MAX_WBITS)
        html = unzip.decode('utf8')
        
        # получение url-адресов изображений и количества страниц с изображениями
        if 'checkcaptcha' in html:
            print('Achtung CAPTCHA! Game Over!')
            return ['captcha', 'captcha']
        else:
            images_urls = re.findall(" <img[^>]*src=[\"|\'](.+?)220x330", html)
            images = ';;'.join([i[50:] for i in images_urls]) + ';;'
            
            nums = re.findall(" <li >[^>]{1,}>(.+?)<", html)
            pages = ';'.join([x for i, x in enumerate(nums) if i == nums.index(x)])
            
            print('Id: %s, количество дополнительных изображений: %s' % (name, str(len(images_urls))))
            return [images, pages]
    except urllib.error.HTTPError as exception:
        print('Id: %s, ошибка 404: Страница не найдена!' % name)
        return [np.nan, np.nan]

In [ ]:
df_new['Img_urls'] = np.nan
df_new['Img_pages'] = np.nan
df_new.head()

In [ ]:
# Загрузка url-адресов дополнительных изображений знаменитостей для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
#for i in notebook.tqdm(df_new.query('Img_urls.isnull() & Role=="актер"', engine='python').index):
for i in notebook.tqdm(df_new.query('Eng_name.isnull() & Img_urls.isnull() & Role=="актер"', engine='python').index):
    add_res = get_additional_images_urls(df_new.loc[i, 'Kpoisk_id'])
         
    # остановка цикла, если кинопоиск стал выбрасывать капчу
    if add_res[0] == 'captcha' or add_res[1] == 'captcha':
        break
    # добавление полученных данных в датафрейм
    try:
        if pd.isnull(df_new.loc[i, 'Img_urls']):
            df_new.loc[i, 'Img_urls'] = add_res[0]
        else:
            df_new.loc[i, 'Img_urls'] = f"{df_new.loc[i, 'Img_urls']}" + add_res[0]
        
        df_new.loc[i, 'Img_pages'] = add_res[1]
        
        cnt += 1
        # через каждые 5 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 5 == 0:
            df_new.to_csv(df_path + '/celebs_final.csv')
    except TypeError:
        print('Error get data!')

    time.sleep(random.randrange(2, 5))

In [ ]:
# Загрузка url-адресов дополнительных изображений знаменитостей для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
for i in notebook.tqdm(df_new.query('Img_urls.isnull() & Role=="актриса"', engine='python').index):
    add_res = get_additional_images_urls(df_new.loc[i, 'Kpoisk_id'])
         
    # остановка цикла, если кинопоиск стал выбрасывать капчу
    if add_res[0] == 'captcha' or add_res[1] == 'captcha':
        break
    # добавление полученных данных в датафрейм
    try:
        if pd.isnull(df_new.loc[i, 'Img_urls']):
            df_new.loc[i, 'Img_urls'] = add_res[0]
        else:
            df_new.loc[i, 'Img_urls'] = f"{df_new.loc[i, 'Img_urls']}" + add_res[0]
        
        df_new.loc[i, 'Img_pages'] = add_res[1]
        
        cnt += 1
        # через каждые 5 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 5 == 0:
            df_new.to_csv(df_path + '/celebs_final.csv')
    except TypeError:
        print('Error get data!')
        
    time.sleep(random.randrange(7, 19))

In [ ]:
df_new.head()

In [ ]:
df_new.to_csv(df_path + '/celebs_final.csv')

In [ ]:
df_new.query('Img_pages!="" & Img_pages.notnull() & Eng_name=="Odette Annable"', engine='python')

Посмотрим на то, что мы получили после парсинга первых страниц с галереями изображений знаменитостей. Знаменитостей, у которых только две страницы:

In [ ]:
df_new.query('Img_pages=="2"', engine='python').shape[0]

Знаменитостей, у который страниц в галереи изображений три:

In [ ]:
df_new.query('Img_pages=="2;3"', engine='python').shape[0]

Знаменитостей, у которых страниц в галерее изображений четыре и более:

In [ ]:
df_new.query('Img_pages=="2;3;4"', engine='python').shape[0]

In [ ]:
df_new.query('Img_pages!="" & Img_pages.notnull()', engine='python').index

Загрузка URL-адресов дополнительных изображений знаменитостей со 2 страницы для их последующего парсинга с сайта Kinopoisk.ru.

In [ ]:
# Загрузка url-адресов дополнительных изображений знаменитостей со 2 страницы для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
for i in notebook.tqdm(df_new.query('Img_pages!="" & Img_pages.notnull()', engine='python').index):
    add_res = get_additional_images_urls(df_new.loc[i, 'Kpoisk_id'], p=2)
         
    # остановка цикла, если кинопоиск стал выбрасывать капчу
    if add_res[0] == 'captcha' or add_res[1] == 'captcha':
        break
    # добавление полученных данных в датафрейм
    try:
        if pd.isnull(df_new.loc[i, 'Img_urls']):
            df_new.loc[i, 'Img_urls'] = add_res[0]
        else:
            df_new.loc[i, 'Img_urls'] = f"{df_new.loc[i, 'Img_urls']}" + add_res[0]
        
        #df_new.loc[i, 'Img_pages'] = add_res[1]
        
        cnt += 1
        # через каждые 5 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 5 == 0:
            df_new.to_csv(df_path + '/celebs_final.csv')
    except TypeError:
        print('Error get data!')

    time.sleep(random.randrange(7, 15))

In [ ]:
df_new.loc[1, 'Img_urls']

In [ ]:
df_new.loc[1, 'Img_pages']

In [ ]:
df_new.to_csv(df_path + '/celebs_final.csv')

Загрузка URL-адресов дополнительных изображений знаменитостей с 3 страницы для их последующего парсинга с сайта Kinopoisk.ru.

In [ ]:
# Загрузка url-адресов дополнительных изображений знаменитостей с 3 страницы для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
for i in notebook.tqdm(df_new.query('Img_pages=="2;3"', engine='python').index):
    add_res = get_additional_images_urls(df_new.loc[i, 'Kpoisk_id'], p=3)
         
    # остановка цикла, если кинопоиск стал выбрасывать капчу
    if add_res[0] == 'captcha' or add_res[1] == 'captcha':
        break
    # добавление полученных данных в датафрейм
    try:
        if pd.isnull(df_new.loc[i, 'Img_urls']):
            df_new.loc[i, 'Img_urls'] = add_res[0]
        else:
            df_new.loc[i, 'Img_urls'] = f"{df_new.loc[i, 'Img_urls']}" + add_res[0]
        
        #df_new.loc[i, 'Img_pages'] = add_res[1]
        
        cnt += 1
        # через каждые 5 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 5 == 0:
            df_new.to_csv(df_path + '/celebs_final.csv')
    except TypeError:
        print('Error get data!')

    time.sleep(random.randrange(7, 15))

Загрузка URL-адресов дополнительных изображений знаменитостей с 4 страницы для их последующего парсинга с сайта Kinopoisk.ru.

In [ ]:
df_new.to_csv(df_path + '/celebs_final.csv')

In [ ]:
# Загрузка url-адресов дополнительных изображений знаменитостей с 4 страницы для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
for i in notebook.tqdm(df_new.query('Img_pages=="2;3;4"', engine='python').index):
    add_res = get_additional_images_urls(df_new.loc[i, 'Kpoisk_id'], p=4)
         
    # остановка цикла, если кинопоиск стал выбрасывать капчу
    if add_res[0] == 'captcha' or add_res[1] == 'captcha':
        break
    # добавление полученных данных в датафрейм
    try:
        if pd.isnull(df_new.loc[i, 'Img_urls']):
            df_new.loc[i, 'Img_urls'] = add_res[0]
        else:
            df_new.loc[i, 'Img_urls'] = f"{df_new.loc[i, 'Img_urls']}" + add_res[0]
        
        # добавление новых страниц
        add_nums = add_res[1].split(';')
        nums = df_new.loc[i, 'Img_pages'].split(';')
        new_nums = nums + add_nums
        new_nums.sort()
        uni_pages = ';'.join([x for i, x in enumerate(new_nums) if i == new_nums.index(x)])
        df_new.loc[i, 'Img_pages'] = uni_pages
        
        cnt += 1
        # через каждые 5 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 5 == 0:
            df_new.to_csv(df_path + '/celebs_final.csv')
    except TypeError:
        print('Error get data!')

    time.sleep(random.randrange(7, 15))

In [ ]:
df_new.query('Img_pages=="2;3;4"', engine='python').index

In [ ]:
df_new.to_csv(df_path + '/celebs_final.csv')

Посмотрим на то, у скольки знаменитостей дополнительные изображения завершаются 5 страницей:

In [ ]:
df_new.query('Img_pages=="1;2;3;4;5"', engine='python').shape[0]

Посмотрим на то, у скольки знаменитостей дополнительные изображения завершаются 6 страницей:

In [ ]:
df_new.query('Img_pages=="1;2;3;4;5;6"', engine='python').shape[0]

Посмотрим на то, у скольки знаменитостей дополнительные изображения завершаются 7 страницей:

In [ ]:
df_new.query('Img_pages=="1;2;3;4;5;6;7"', engine='python').shape[0]

Загрузка URL-адресов дополнительных изображений знаменитостей с 5 страницы для их последующего парсинга с сайта Kinopoisk.ru.

In [ ]:
# Загрузка url-адресов дополнительных изображений знаменитостей с 5 страницы для их последующего парсинга с сайта kinpoisk.ru

cnt = 0
for i in notebook.tqdm(df_new.query('Img_pages=="1;10;11;12;13;14;15;16;2;3;4;5;6;7;8;9"', engine='python').index): # нужно менять строку запроса тут
    add_res = get_additional_images_urls(df_new.loc[i, 'Kpoisk_id'], p=16) # нужно менять номер страницы тут
         
    # остановка цикла, если кинопоиск стал выбрасывать капчу
    if add_res[0] == 'captcha' or add_res[1] == 'captcha':
        break
    # добавление полученных данных в датафрейм
    try:
        if pd.isnull(df_new.loc[i, 'Img_urls']):
            df_new.loc[i, 'Img_urls'] = add_res[0]
        else:
            df_new.loc[i, 'Img_urls'] = f"{df_new.loc[i, 'Img_urls']}" + add_res[0]
        
        # добавление новых страниц
        add_nums = add_res[1].split(';')
        nums = df_new.loc[i, 'Img_pages'].split(';')
        new_nums = nums + add_nums
        new_nums.sort()
        uni_pages = ';'.join([x for i, x in enumerate(new_nums) if i == new_nums.index(x)])
        #df_new.loc[i, 'Img_pages'] = uni_pages # отключать комментирование для последней страницы и включать для остальных
        
        cnt += 1
        # через каждые 5 запросов сохрняем результаты в файл, чтобы не потерять
        if cnt % 5 == 0:
            df_new.to_csv(df_path + '/celebs_final.csv')
    except TypeError:
        print('Error get data!')

    time.sleep(random.randrange(2, 5))

Аналогичным образом парсим вглубь базу данных Кинопоиска пока не будут пройдены все доступные для каждой знаменитости страницы. 

In [ ]:
df_new.query('Eng_name=="Angelina Jolie"', engine='python')

Теперь для актрисы Анджелины Джоли может быть загружено следующее количество дополнительных изображений:

In [ ]:
len(set(df_new.loc[142, 'Img_urls'].split(';;')))

Всего для всех актёров и актрис может быть получено слещующее количество дополнительных изображений:

In [ ]:
def counter(x):
    return len(set(x.split(';;')))

df_new[df_new['Img_urls'].notnull()]['Img_urls'].apply(lambda x: counter(x)).sum()

In [ ]:
df_new.query('Img_urls.notnull()', engine='python').shape[0]

In [ ]:
df_new.query('Img_urls.notnull()', engine='python').index

In [ ]:
# функция загрузки дополнительных изображений знаменитостей с сайта kinopoisk

def get_additional_images(df, gender='men', natio='eng'):
    df_path = '//192.168.1.113/Public/Documents/PET-project'
    
    # проверяем наличие папки gallery и при необходимости создаём её
    if os.path.exists(f'{df_path}/images/') != True:
        os.mkdir(f'{df_path}/images/')
        
    # проверяем наличие подпапки разделения по полу и при необходимости создаём её
    if os.path.exists(f'{df_path}/images/{gender}') != True:
        os.mkdir(f'{df_path}/images/{gender}')
        
    # проверяем наличие подпапки разделения по признаку отечественный/иностранный и при необходимости создаём её
    if os.path.exists(f'{df_path}/images/{gender}/{natio}') != True:
        os.mkdir(f'{df_path}/images/{gender}/{natio}')
    
    if natio == 'eng':
        req = 'Eng_name.notnull()'
        name = 'Eng_name'
    if natio == 'rus':
        req = 'Eng_name.isnull()'
        name = 'Rus_name'
        
    if gender == 'men':
        role = 'актер'
    if gender == 'women':
        role = 'актриса'
        
    # пройдемся по каждому имени
    for face in notebook.tqdm(list(df.query(f'{req} & Img_urls.notnull() & Role=="{role}"',\
                                            engine='python')[f'{name}'].values)):
        face = face.strip()
        
        if os.path.exists(f'{df_path}/images/{gender}/{natio}/{face}') != True:
            os.mkdir(f'{df_path}/images/{gender}/{natio}/{face}')
        else:
            print(f'Images for {face} already uploaded!')
            continue
            
        print('Start parse images for %s!' % face)
        
        ind = df.query(f'{name}=="{face}"').index
        
        try:
            img_urls_str = df_new.loc[ind[0], 'Img_urls']
            
            img_urls = list(set(img_urls_str.split(';;')))[:]
            
            i = 0
            for img_url in img_urls:
                if len(img_url) > 1:
                    i += 1
                    # Загрузка исходного кода страницы и получение ссылок на изображения
                    resp = requests.get(f'https://avatars.mds.yandex.net/get-kinopoisk-image{img_url}1920x')
                    
                    with open(f'{df_path}/images/{gender}/{natio}/{face}/Image_{i}.jpg', 'wb') as f:
                        try:
                            f.write(resp.content)
                            print('    :: parsed image :: %s!' % img_url)
                        except OSError:
                            print('    :: Error parse :: %s (!)' % img_url)
                            continue
        except (IndexError, AttributeError):
            print('Index error for %s (!)' % face)

In [ ]:
# загрузка дополнительных изображений иностранных актёров
get_additional_images(df_new)

In [ ]:
# загрузка дополнительных изображений иностранных актрис
get_additional_images(df_new, gender='women')

In [ ]:
# загрузка дополнительных изображений отечественных актёров
get_additional_images(df_new, natio='rus')

In [ ]:
# загрузка дополнительных изображений отечественных актрис
get_additional_images(df_new, gender='women', natio='rus')

После загрузки фотографий знаменитостей с сайта Кинопоиск необходимо произвести уже знакомые операции по выделению лиц на изображениях и их последующего сохранения с масштабированием.

In [ ]:
df_path = 'D://PET-project'

In [ ]:
# функция обрезки изображений знаменитостей из папки images для сохранения фрагментов с лицами

def image_compress(gender='men'):
    nation = ['eng', 'rus']
    for natio in nation:
        folders = glob.glob(f'{df_path}/images/{gender}/{natio}/*')
        
        # пройдемся по каждому имени
        for face in notebook.tqdm(folders):
            face = face.strip()
            
            # выгрузим все название файлов из папки
            files = glob.glob(f'{face}/*')
            
            # пройдемся по списку файлов в цикле
            for img_file in files:
                try:
                    # обрезка изображений
                    image_cropper(img_file)
                    print('[%] Image: {} succefull cropped!\n'.format(img_file))
                except:
                    print('[!] Error crop image: {}\n'.format(img_file))
                    continue

In [ ]:
# функция сжатия/расширение изображений знаменитостей в папке images

def images_gallery_compress(gender='men'):
    nation = ['eng', 'rus']
    for natio in nation:
        folders = glob.glob(f'{df_path}/images/{gender}/{natio}/*')
        
        for face in notebook.tqdm(folders):
            face = face.strip()
            
            # выгрузим все названия файлов из подпапок знаменитостей
            files = glob.glob(f'{face}/*')
            
            for img_file in files:
                # обрезка изображений
                try:
                    with Image.open(img_file) as image:
                        size = image.size 
                        
                        # определим его наименьшую сторону
                        if size[0] < size[1]:
                            min_size = size[0]
                        else:
                            min_size = size[1]
                            
                        if min_size > 192:
                            # получим коэффициент, на который нужно уменьшить/увеличить
                            # изображение по одной из сторон до 256
                            coef = SIZE / min_size
                            
                            # изменяем размер изображения
                            resized_image = image.resize((int(size[0] * coef), int(size[1] * coef)))
                            resized_image = resized_image.convert('RGB')
                            # сохраняем изображение в новом размере
                            resized_image.save(img_file)
                    if min_size < 192:
                        os.remove(img_file)
                        print('[%] Image: {} delete!'.format(img_file))
                except:
                    os.remove(img_file)
                    print('Error in file: %s!' % img_file)

In [ ]:
# обрезка галлереи изображений актёров
image_compress()

In [ ]:
# обрезка галлереи изображений актрис
image_compress(gender='women')

In [ ]:
# сжатие/растяжение галлереи изображений актёров
images_gallery_compress()

In [ ]:
# сжатие/растяжение галлереи изображений актрис
images_gallery_compress(gender='women')

После того как загружены изображения, можно произвести отбор наилучших изображений для каждой знаменистости с целью подготовки обучающих датасетов для обучения моделей.